<a href="https://colab.research.google.com/github/Sarthakj0805/Image_Classification_Model/blob/master/Img_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# importing all the required libraries

import tensorflow as tf
import tensorboard
import sys
from matplotlib import pyplot as plt
import seaborn as sns
from datetime import datetime
from packaging import version
import numpy as np
from tensorflow.keras import utils as np_utils
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam


In [13]:
# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = cifar10.load_data()
	# one hot encode target values
	trainY = tf.keras.utils.to_categorical(trainY)
	testY = tf.keras.utils.to_categorical(testY)
	return trainX, trainY, testX, testY


In [ ]:
# summarize loaded dataset
def summarize_data():
  trainX, trainY, testX, testY = load_dataset()	
  print('Train: X=%s, y=%s' % (trainX.shape, trainY.shape))
  print('Test: X=%s, y=%s' % (testX.shape, testY.shape))
  # plot first few images
  for i in range(9):
    # define subplot
    plt.subplot(330 + 1 + i)
    # plot raw pixel data
    plt.imshow(trainX[i])
  # show the figure
  plt.show()
  plt.close()

In [14]:
# normalize data
def normalize(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm - train_norm.min() / train_norm.max() - train_norm.min() 
	test_norm = test_norm - test_norm.min() / test_norm.max() - test_norm.min()
	# return normalized images
	return train_norm, test_norm
	

In [ ]:
# define the cnn model
def define_model():
  model = Sequential()
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
  model.add(BatchNormalization())
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.2))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(BatchNormalization())
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.3))
  model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(BatchNormalization())
  model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.4))
  model.add(Flatten())
  model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
  model.add(BatchNormalization())
  model.add(Dropout(0.5))
  model.add(BatchNormalization())
  model.add(Dense(10, activation='softmax'))
  # compile model
  opt = SGD(learning_rate=0.005, momentum=0.9)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  # print(model.summary())
  return model

In [ ]:
# plot the learning curves for loss and accuracy metrics
def summary_plots(history):
	sns.set(context='notebook', palette='bright', style='dark')
	# plot loss
	plt.subplot(211)
	plt.title('Cross Entropy Loss')
	plt.xlabel('epoch')
	plt.ylabel('loss')
	plt.plot(history.history['loss'], color='green', label='train')
	plt.plot(history.history['val_loss'], color='red', label='test')
	# plot accuracy
	plt.subplot(212)
	plt.title('Classification Accuracy')
	plt.xlabel('epoch')
	plt.ylabel('accuracy')
	plt.plot(history.history['accuracy'], color='green', label='train')
	plt.plot(history.history['val_accuracy'], color='red', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	plt.savefig(filename + '_plot.png')
	plt.show()
	plt.close()

In [ ]:
# run the test harness for evaluating a model
def run_model():
	# load dataset
	trainX, trainY, testX, testY = load_dataset()
	# normalize data
	trainX, testX = normalize(trainX, testX)
	# define model
	model = define_model()
	# create data generator
	datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
	# prepare iterator
	it_train = datagen.flow(trainX, trainY, batch_size=64)
	#define the keras tensorboard callback
	logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
	tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
	# fit model
	# steps = int(trainX.shape[0] / 128)
	history = model.fit(it_train, steps_per_epoch=300, epochs=100, validation_data=(testX, testY), verbose=1, callbacks=[tensorboard_callback])
	# evaluate model
	_, acc = model.evaluate(testX, testY, verbose=1)
	print('\n \n \n')
	print('> %.3f' % (acc * 100.0), "\n \n \n")
	# learning curves
	summary_plots(history)
  # save model
	# uncomment to save the model
	# model.save('path to folder')

# entry point, run the test harness
run_model()

In [ ]:
''' to check the model using predictions on the test dataset using the saved model.pb file'''

# load dataset
trainX, trainY, testX, testY = load_dataset()

# normalize data
trainX, testX = normalize(trainX, testX)

#enter any index between 0-10000 for the test image
n = int(input("enter n: "))
img = testX[n]

# reshape into a single sample with 3 channels
img = img.reshape(1, 32, 32, 3)

#enter the path to the saved model folder to load the model
model = load_model('path to saved model.pb folder')

classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck' ]
result = model.predict(img)

print('the selected image is: \n')
plt.imshow(testX[n].astype('uint8'))
plt.show()
plt.close()

print("the predicted class is : ", classes[np.argmax(result)], "\n")
print("the correct class is : " , classes[np.argmax(testY[n])], "\n")

In [ ]:
'''test the model on sample images using saved model.pb file'''
# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, target_size=(32, 32))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 32, 32, 3)
	# prepare pixel data
	img = img.astype('float32')
	img = img - img.min() / img.max() - img.min()
	return img

#enter the path to the saved model folder to load the model
model = load_model('path to saved model folder')

classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck' ]

sample_img = ['cat.jpg', 'dog.png', 'airplane.jpg']
for i in sample_img:
  print(i)
  #enter the path to the saved Assets folder to load the images
  img = load_image('path to the Assets folder' + str(i))
  result = model.predict(img)

  print('the selected image is: \n')
  plt.imshow(img.reshape(32,32,3).astype('uint8'))
  plt.show()
  plt.close()

  print("the predicted class is : ", classes[np.argmax(result)], '\n')